In [1]:
import os
import sys

# Allowing Jupyter to load modules and clases from the file system
module_path = os.path.abspath(os.path.join("../src"))
if module_path not in sys.path:
    sys.path.append(module_path)

### RAG Triad of Metrics

In [2]:
import warnings

warnings.filterwarnings("ignore")

In [3]:
import os

import openai
import utils

openai.api_key = utils.get_openai_api_key()

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [4]:
from trulens_eval import Tru

tru = Tru()
tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


In [5]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["../data/eBook-How-to-Build-a-Career-in-AI.pdf"]
).load_data()

In [6]:
from llama_index.core import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

In [7]:
# from llama_index.llms import OpenAI
# llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)
from llama_index.llms.azure_openai import AzureOpenAI
from utils import build_sentence_window_index


sentence_index = build_sentence_window_index(
    document, llm, embed_model="local:BAAI/bge-small-en-v1.5", save_dir="sentence_index"
)

In [8]:
from utils import get_sentence_window_query_engine

sentence_window_engine = get_sentence_window_query_engine(
    llm,
    sentence_index,
)

In [9]:
output = sentence_window_engine.query("How do you create your AI portfolio?")
output.response

'To create an AI portfolio, start with small projects to build foundational skills and gradually progress to more complex ones. Focus on demonstrating skill progression over time. Clearly communicate the value and outcomes of each project to showcase your ability to explain your work. This will help in gaining trust and resources for larger projects. Additionally, highlight your leadership skills, especially in teamwork settings, to show your capability in steering projects with technical insights.'

### Feedback Functions

In [10]:
import nest_asyncio

nest_asyncio.apply()

In [11]:
from trulens_eval.feedback.provider.openai import OpenAI

provider = OpenAI()

#### 1. Answer relevance

In [12]:
from trulens_eval import Feedback

f_qa_relevance = Feedback(
    provider.relevance_with_cot_reasons,
    name="Answer Relevance",
).on_input_output()

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .


#### 2. Context Relevance

In [13]:
from trulens_eval import TruLlama

context_selection = TruLlama.select_source_nodes().node.text

In [14]:
import numpy as np

f_qs_relevance = (
    Feedback(provider.qs_relevance, name="Context Relevance")  # Provides conte
    .on_input()  # Pointer to user query
    .on(context_selection)  # Pointer to retrieved contexts (intermediate results)
    .aggregate(np.mean)  # Aggregate scores across all retrieved context
)

✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input context will be set to __record__.app.query.rets.source_nodes[:].node.text .


In [15]:
import numpy as np

f_qs_relevance = (
    Feedback(
        provider.qs_relevance_with_cot_reasons,  # Provides context relevance with chain of thoughts
        name="Context Relevance",
    )
    .on_input()
    .on(context_selection)
    .aggregate(np.mean)
)

✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input context will be set to __record__.app.query.rets.source_nodes[:].node.text .


#### 3. Groundedness

In [16]:
f_groundedness = (
    Feedback(provider.groundedness_measure_with_cot_reasons, name="Groundedness")
    .on(context_selection)
    .on_output()
)

✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


### Evaluation of the RAG application

In [17]:
from trulens_eval import TruLlama
from trulens_eval import FeedbackMode

tru_recorder = TruLlama(
    sentence_window_engine,
    app_id="App_1",
    feedbacks=[f_qa_relevance, f_qs_relevance, f_groundedness],
)

In [18]:
eval_questions = []
with open("../data/eval_questions.txt", "r") as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        eval_questions.append(item)

In [19]:
eval_questions

['What are the keys to building a career in AI?',
 'How can teamwork contribute to success in AI?',
 'What is the importance of networking in AI?',
 'What are some good habits to develop for a successful career?',
 'How can altruism be beneficial in building a career?',
 'What is imposter syndrome and how does it relate to AI?',
 'Who are some accomplished individuals who have experienced imposter syndrome?',
 'What is the first step to becoming good at AI?',
 'What are some common challenges in AI?',
 'Is it normal to find parts of AI challenging?']

In [20]:
eval_questions.append("How can I be successful in AI?")

In [21]:
eval_questions

['What are the keys to building a career in AI?',
 'How can teamwork contribute to success in AI?',
 'What is the importance of networking in AI?',
 'What are some good habits to develop for a successful career?',
 'How can altruism be beneficial in building a career?',
 'What is imposter syndrome and how does it relate to AI?',
 'Who are some accomplished individuals who have experienced imposter syndrome?',
 'What is the first step to becoming good at AI?',
 'What are some common challenges in AI?',
 'Is it normal to find parts of AI challenging?',
 'How can I be successful in AI?']

In [22]:
for question in eval_questions:
    with tru_recorder as recording:
        sentence_window_engine.query(question)

In [23]:
records, feedback = tru.get_records_and_feedback(app_ids=[])
records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,latency,total_tokens,total_cost
0,App_1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_a55f2c25e4fdf45f5ab890dd00c9fceb,"""What are the keys to building a career in AI?""","""The keys to building a career in AI include f...",-,"{""record_id"": ""record_hash_a55f2c25e4fdf45f5ab...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-09-26T15:13:40.152558"", ""...",2024-09-26T15:13:42.276732,2,1721,0.009565
1,App_1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_5c2b2903c6ce285910436f5923be3b65,"""How can teamwork contribute to success in AI?""","""Teamwork contributes to success in AI by enab...",-,"{""record_id"": ""record_hash_5c2b2903c6ce2859104...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-09-26T15:13:42.559714"", ""...",2024-09-26T15:13:44.659170,2,1656,0.008920
2,App_1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_4fd438faab493dbb564425f2319ae46f,"""What is the importance of networking in AI?""","""Networking in AI is crucial as it helps build...",-,"{""record_id"": ""record_hash_4fd438faab493dbb564...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-09-26T15:13:44.899434"", ""...",2024-09-26T15:13:46.553361,1,1680,0.009070
3,App_1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_d93639f6c7d5830a90c085e06b0ef272,"""What are some good habits to develop for a su...","""Developing good habits in areas such as eatin...",-,"{""record_id"": ""record_hash_d93639f6c7d5830a90c...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-09-26T15:13:46.809731"", ""...",2024-09-26T15:13:48.224622,1,1657,0.008715
4,App_1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_e89a10cc75bde68aec1edd4cafa3cc19,"""How can altruism be beneficial in building a ...","""Altruism can be beneficial in building a care...",-,"{""record_id"": ""record_hash_e89a10cc75bde68aec1...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-09-26T15:13:48.498585"", ""...",2024-09-26T15:13:49.969123,1,1645,0.008695


In [24]:
import pandas as pd

pd.set_option("display.max_colwidth", None)
records[["input", "output"] + feedback]

,input,output
0,"""What are the keys to building a career in AI?""","""The keys to building a career in AI include focusing on teamwork, developing a strong professional network, conducting an effective job search, maintaining personal discipline, and practicing altruism. Teamwork involves collaborating and communicating effectively with others. Networking is about building a supportive community rather than just personal connections. The job search should be approached positively, without adversarial attitudes. Personal discipline involves cultivating good habits in various aspects of life. Lastly, altruism is about helping others while advancing your own career."""
1,"""How can teamwork contribute to success in AI?""","""Teamwork contributes to success in AI by enabling collaboration, where individuals can influence and be influenced by others. This collaboration enhances problem-solving and project outcomes, as working in teams often leads to better results than working individually. Interpersonal and communication skills are crucial in this process, allowing team members to effectively share ideas and insights."""
2,"""What is the importance of networking in AI?""","""Networking in AI is crucial as it helps build a strong professional community, which can provide support, advice, and opportunities. Instead of focusing solely on personal networking, contributing to and building up communities can lead to meeting more people and forming genuine friendships. This community-oriented approach can be particularly beneficial for introverts who may find traditional networking challenging."""
3,"""What are some good habits to develop for a successful career?""","""Developing good habits in areas such as eating, exercise, sleep, personal relationships, work, learning, and self-care can significantly contribute to a successful career. These habits help maintain health and facilitate progress over time."""
4,"""How can altruism be beneficial in building a career?""","""Altruism can be beneficial in building a career by fostering better outcomes for oneself while helping others. By aiming to lift others during your journey, you create a supportive network and positive relationships, which can lead to mutual success and opportunities."""
5,"""What is imposter syndrome and how does it relate to AI?""","""Imposter syndrome is a feeling of self-doubt and insecurity where individuals, despite their success, feel like frauds and question their belonging in a community. In the context of AI, newcomers and even accomplished individuals may experience this due to the field's technical complexity and the presence of highly capable people. It's important to recognize that many successful figures have experienced imposter syndrome and that struggling with challenges is a common part of growth in AI. Encouragement and support within the community can help individuals overcome these feelings."""
6,"""Who are some accomplished individuals who have experienced imposter syndrome?""","""Some accomplished individuals who have experienced imposter syndrome include Sheryl Sandberg, Michelle Obama, Tom Hanks, and Mike Cannon-Brookes."""
7,"""What is the first step to becoming good at AI?""","""The first step to becoming good at AI is learning foundational technical skills. This includes understanding key machine learning models and concepts, as well as gaining knowledge in deep learning and software development."""
8,"""What are some common challenges in AI?""","""Some common challenges in AI include experiencing imposter syndrome, dealing with the technical complexity of the field, and overcoming initial difficulties in understanding concepts like linear and logistic regression or debugging neural networks. Additionally, reading and comprehending complex research papers can be challenging even for experienced individuals."""
9,"""Is it normal to find parts of AI challenging?""","""Yes, it's normal to find parts of AI challenging. Many people, even those wh

In [25]:
tru.get_leaderboard(app_ids=[])

,latency,total_cost
app_id,,
App_1,5.181818,0.0103


In [26]:
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.50.114:63026 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>